In [1]:
import pandas as pd
import numpy as np

In [2]:
#연령 그룹화: (0,10) (20) (30) (40,50) (60) (70,80)
korean_df = pd.read_csv("./월별데이터 정리/month_korean.csv")

In [3]:
korean_df = korean_df.drop(columns=['emd_cd'])

In [4]:
korean_df['resd'].value_counts()

제주    21672
그외    21672
Name: resd, dtype: int64

In [5]:
korean_df['m'] = pd.to_datetime(korean_df['m']) #datetime type
korean_df['avg_resd'] = korean_df['sum_resd_pop_cnt']/korean_df['count_num']
korean_df['avg_work'] = korean_df['sum_work_pop_cnt']/korean_df['count_num']
korean_df['avg_visit'] = korean_df['sum_visit_pop_cnt']/korean_df['count_num']

In [6]:
# 방문객 중에는 거주자가 없음을 확인
korean_df[korean_df['resd']=="그외"]['sum_resd_pop_cnt'].unique()

array([0.])

In [7]:
korean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43344 entries, 0 to 43343
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   m                  43344 non-null  datetime64[ns]
 1   resd               43344 non-null  object        
 2   sex                43344 non-null  object        
 3   age                43344 non-null  int64         
 4   emd_nm             43344 non-null  object        
 5   count_num          43344 non-null  int64         
 6   sum_resd_pop_cnt   43344 non-null  float64       
 7   sum_work_pop_cnt   43344 non-null  float64       
 8   sum_visit_pop_cnt  43344 non-null  float64       
 9   avg_resd           43344 non-null  float64       
 10  avg_work           43344 non-null  float64       
 11  avg_visit          43344 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(3)
memory usage: 4.0+ MB


In [8]:
## 새로운 테이블 생성

# 성별을 담은 리스트
sex_list = list(korean_df['sex'].unique())

# 연령대를 담은 리스트
age_list = list(korean_df['age'].unique())

# 거주 행정 구역을 담은 리스트(여행객, 거주민)
resd_list = list(korean_df['resd'].unique())

# 각 동의 이름을 담은 리스트 생성
region_list = list(korean_df['emd_nm'].unique())

# 2018-01-01 ~ 2021-06-01 (주기: 월) 날짜에 동별 정보를 담은 dataframe
dt_index = pd.date_range(start='20180101', end='20210601', freq='MS')
val1 = list(dt_index)*len(region_list)
val2 = []
for i in region_list:
    for j in range(0,len(dt_index)):
        val2.append(i)
        
date_df = pd.DataFrame({'Year_Month':val1, 'emd_nm':val2})

In [9]:
age_list

[0, 20, 30, 40, 60, 70]

In [11]:
avg_resd_list = list(np.array(korean_df['avg_resd'].tolist()))
avg_work_list = list(np.array(korean_df['avg_work'].tolist()))
avg_visit_list = list(np.array(korean_df['avg_visit'].tolist()))


# 거주여부별,성별 연령별(0,20,30,40,60,70)
# 거주 0 : 성별, 연령별, 특징(3) (2*6*3 = 36) 
# 거주 X : 성별, 연령별, 특징(2) (2*6*2 = 24)
# => 60개의 변수 => 더 줄여야됨. 

#거주 0
df_array1 = np.zeros(shape=(len(date_df),len(sex_list)*len(age_list)*3))

# 거주 X
df_array2 = np.zeros(shape=(len(date_df),len(sex_list)*len(age_list)*2))


# 거주 0
for i in range(0,len(date_df)):
    date = date_df["Year_Month"][i]
    emd = date_df["emd_nm"][i]
    for j in range(0,len(sex_list)):
        sex = sex_list[j%2]
        for k in range(0,len(age_list)):
            age = age_list[k%len(age_list)]
            colnum = j*18+k*3
            sample_list = korean_df[(korean_df['m'] == date)
                                   &(korean_df['emd_nm'] == emd)
                                   &(korean_df['sex'] == sex)
                                   &(korean_df['age'] == age)
                                   &(korean_df['resd'] == "제주")].index.tolist()
            if sample_list == []:
                df_array1[i,colnum] = np.NaN
                df_array1[i,colnum+1] = np.NaN
                df_array1[i,colnum+2] = np.NaN
            else:
                df_array1[i,colnum] = avg_resd_list[sample_list[0]]           #거주인구 평균
                df_array1[i,colnum+1] = avg_work_list[sample_list[0]]   #근무인구 평균
                df_array1[i,colnum+2] = avg_visit_list[sample_list[0]]   #그외인구 평균
                    

In [12]:
# 거주 X
for i in range(0,len(date_df)):
    date = date_df["Year_Month"][i]
    emd = date_df["emd_nm"][i]
    for j in range(0,len(sex_list)):
        sex = sex_list[j%2]
        for k in range(0,len(age_list)):
            age = age_list[k%len(age_list)]
            colnum = j*12+k*2
            sample_list = korean_df[(korean_df['m'] == date)
                                   &(korean_df['emd_nm'] == emd)
                                   &(korean_df['sex'] == sex)
                                   &(korean_df['age'] == age)
                                   &(korean_df['resd'] == "그외")].index.tolist()
            if sample_list == []:
                df_array2[i,colnum] = np.NaN
                df_array2[i,colnum+1] = np.NaN
            else:
                df_array2[i,colnum] = avg_work_list[sample_list[0]]           #근무인구 평균
                df_array2[i,colnum+1] = avg_visit_list[sample_list[0]]   #그외인구 평균

In [13]:
df_array1

array([[350.44944106,   8.01049012, 193.04540168, ..., 167.44905934,
          1.26634254,  96.54519785],
       [376.89886682,  11.89272091, 211.79849122, ..., 170.19927865,
          1.24393966,  99.42507158],
       [302.15731284,  12.98029288, 173.54502634, ..., 156.50776794,
         14.88693999,  87.13105921],
       ...,
       [180.54673264,   9.32490139,  25.92475889, ..., 151.98240722,
          2.64811632,  22.05018806],
       [203.19829321,   4.64044926,  31.3334627 , ..., 140.32804019,
          0.61087359,  27.90842735],
       [190.30075014,   5.43191764,  36.21221139, ..., 148.26343597,
          0.        ,  26.02787931]])

In [14]:
df_array2

array([[  4.63635235, 113.01872594,   8.70849516, ...,  70.21986841,
          5.01554735,  22.29068821],
       [  0.15595126,  90.41866429,   4.73658259, ...,  71.22246875,
          0.        ,  27.66399552],
       [  2.77253279,  34.869517  ,   6.74591559, ...,  99.42732151,
          7.99286012,  32.84886296],
       ...,
       [  0.        ,  13.78654941,   1.11788792, ...,  63.17393403,
          2.37942972,  11.16164646],
       [  0.        ,  18.83391816,   2.31886519, ...,  44.01217567,
          7.10219785,  10.95764711],
       [  0.        ,  19.57976604,   0.15581264, ...,  36.76348778,
          3.57861327,   6.96689923]])

In [26]:
colname_value = "거주O_"+sex_list[i]+"_"+str(age_list[j])


In [27]:
feature_list1 = ["resd","work","visit"]
feature_list2 = ["work","visit"]

colname1 = []
colname2 = []

# 거주 O
for i in sex_list:
    for j in age_list:
        for k in feature_list1:
            colname_value = "거주O_"+i+"_"+str(j)+"_"+k
            colname1.append(colname_value)
            
# 거주 X
for i in sex_list:
    for j in age_list:
        for k in feature_list2:
            colname_value = "거주X_"+i+"_"+str(j)+"_"+k
            colname2.append(colname_value)

In [28]:
# 위에서 만든 각 특징값이 담긴 array배열을 데이터 프레임에 저장
cat_df1 = pd.DataFrame(df_array1, columns = colname1)
cat_df2 = pd.DataFrame(df_array2, columns = colname2)

# date_df, cat_df1, cat_df2 병합
korean_df_new = pd.concat([date_df, cat_df1, cat_df2], axis=1)

In [29]:
korean_df_new

,Year_Month,emd_nm,거주O_남성_0_resd,거주O_남성_0_work,거주O_남성_0_visit,거주O_남성_20_resd,거주O_남성_20_work,거주O_남성_20_visit,거주O_남성_30_resd,거주O_남성_30_work,...,거주X_여성_20_work,거주X_여성_20_visit,거주X_여성_30_work,거주X_여성_30_visit,거주X_여성_40_work,거주X_여성_40_visit,거주X_여성_60_work,거주X_여성_60_visit,거주X_여성_70_work,거주X_여성_70_visit
0,2018-01-01,건입동,350.449441,8.010490,193.045402,297.879964,92.983867,268.254655,419.138116,95.150269,...,0.849652,201.708434,3.095511,149.473093,5.725734,157.793788,0.760684,70.219868,5.015547,22.290688
1,2018-02-01,건입동,376.898867,11.892721,211.798491,304.721008,81.341594,280.109909,419.608918,85.541727,...,3.889460,209.289232,8.019615,144.745262,7.465699,132.971418,5.480244,71.222469,0.000000,27.663996
2,2018-03-01,건입동,302.157313,12.980293,173.545026,273.927896,85.913123,269.133188,417.867035,93.400925,...,0.595223,179.619505,4.454866,145.331568,7.000414,137.050675,1.213751,99.427322,7.992860,32.848863
3,2018-04-01,건입동,292.502968,10.884222,197.994971,284.084417,93.238049,268.335625,401.416736,101.804257,...,4.804073,155.626238,4.202650,150.514100,5.750559,159.957549,6.252506,125.952834,6.015327,46.738731
4,2018-05-01,건입동,282.538264,8.403684,196.990717,295.058321,86.060344,280.138194,401.874829,96.110773,...,5.106274,163.774587,2.476966,143.747018,5.798682,149.727241,0.758502,93.889909,0.000000,32.797218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,2021-02-01,효돈동,222.510444,6.939278,44.244653,142.407419,14.179054,53.808156,180.087750,15.234575,...,1.943441,66.968882,0.000000,49.104046,2.840221,44.647608,2.777232,33.161564,0.383951,13.542675
1802,2021-03-01,효돈동,184.050724,2.127180,34.610074,140.158065,7.427179,44.479943,175.446988,15.988659,...,0.000000,54.151809,3.889325,41.802852,1.498272,42.688094,0.000000,43.031503,0.000000,12.397620
1803,2021-04-01,효돈동,180.546733,9.324901,25.924759,128.872350,7.613932,38.998664,172.078059,16.089038,...,1.168261,61.166594,0.294228,58.343378,1.802332,60.698936,2.620352,63.173934,2.379430,11.161646
1804,2021-05-01,효돈동,203.198293,4.640449,31.333463,129.487713,4.259849,42.330309,167.379332,14.634303,...,0.000000,80.385726,0.000000,67.937546,0.000000,54.486624,0.000000,44.012176,7.102198,10.957647


In [30]:
korean_df_new[korean_df_new["emd_nm"]=="건입동"]

,Year_Month,emd_nm,거주O_남성_0_resd,거주O_남성_0_work,거주O_남성_0_visit,거주O_남성_20_resd,거주O_남성_20_work,거주O_남성_20_visit,거주O_남성_30_resd,거주O_남성_30_work,...,거주X_여성_20_work,거주X_여성_20_visit,거주X_여성_30_work,거주X_여성_30_visit,거주X_여성_40_work,거주X_여성_40_visit,거주X_여성_60_work,거주X_여성_60_visit,거주X_여성_70_work,거주X_여성_70_visit
0,2018-01-01,건입동,350.449441,8.010490,193.045402,297.879964,92.983867,268.254655,419.138116,95.150269,...,0.849652,201.708434,3.095511,149.473093,5.725734,157.793788,0.760684,70.219868,5.015547,22.290688
1,2018-02-01,건입동,376.898867,11.892721,211.798491,304.721008,81.341594,280.109909,419.608918,85.541727,...,3.889460,209.289232,8.019615,144.745262,7.465699,132.971418,5.480244,71.222469,0.000000,27.663996
2,2018-03-01,건입동,302.157313,12.980293,173.545026,273.927896,85.913123,269.133188,417.867035,93.400925,...,0.595223,179.619505,4.454866,145.331568,7.000414,137.050675,1.213751,99.427322,7.992860,32.848863
3,2018-04-01,건입동,292.502968,10.884222,197.994971,284.084417,93.238049,268.335625,401.416736,101.804257,...,4.804073,155.626238,4.202650,150.514100,5.750559,159.957549,6.252506,125.952834,6.015327,46.738731
4,2018-05-01,건입동,282.538264,8.403684,196.990717,295.058321,86.060344,280.138194,401.874829,96.110773,...,5.106274,163.774587,2.476966,143.747018,5.798682,149.727241,0.758502,93.889909,0.000000,32.797218
5,2018-06-01,건입동,279.047919,8.492835,200.508354,283.089141,93.106815,269.266679,384.626055,106.638080,...,5.230187,198.741076,6.673160,165.451189,4.519489,132.968800,2.469978,78.479399,0.000000,30.866637
6,2018-07-01,건입동,296.970009,12.639372,190.511876,276.220382,73.035756,271.902842,379.102953,82.667674,...,5.421124,214.145654,5.534820,129.033589,1.393114,104.977704,0.744103,46.499190,12.194158,18.861765
7,2018-08-01,건입동,338.535896,21.619411,174.609311,271.541591,45.928192,284.045397,382.918883,53.814176,...,3.391427,209.304206,4.048135,157.782203,3.150189,120.251895,6.049909,58.205599,0.000000,37.973258
8,2018-09-01,건입동,301.439389,9.659701,186.581176,286.144033,53.464108,309.556326,383.553958,48.833880,...,4.658037,197.791126,5.948819,168.223350,2.569458,132.520216,2.152015,82.201684,6.044343,39.869170
9,2018-10-01,건입동,305.731892,4.493229,178.777948,314.947146,69.421229,258.907794,374.200796,80.334931,...,3.073476,137.811148,7.317179,134.560363,4.019909,131.855506,4.305446,102.155325,0.000000,29.364054


In [31]:
korean_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 62 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Year_Month       1806 non-null   datetime64[ns]
 1   emd_nm           1806 non-null   object        
 2   거주O_남성_0_resd    1806 non-null   float64       
 3   거주O_남성_0_work    1806 non-null   float64       
 4   거주O_남성_0_visit   1806 non-null   float64       
 5   거주O_남성_20_resd   1806 non-null   float64       
 6   거주O_남성_20_work   1806 non-null   float64       
 7   거주O_남성_20_visit  1806 non-null   float64       
 8   거주O_남성_30_resd   1806 non-null   float64       
 9   거주O_남성_30_work   1806 non-null   float64       
 10  거주O_남성_30_visit  1806 non-null   float64       
 11  거주O_남성_40_resd   1806 non-null   float64       
 12  거주O_남성_40_work   1806 non-null   float64       
 13  거주O_남성_40_visit  1806 non-null   float64       
 14  거주O_남성_60_resd   1806 non-null   float64

In [32]:
korean_df_new.to_pickle("korean_df.pkl")

In [33]:
korean = pd.read_pickle("korean_df.pkl")

In [34]:
korean_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 62 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Year_Month       1806 non-null   datetime64[ns]
 1   emd_nm           1806 non-null   object        
 2   거주O_남성_0_resd    1806 non-null   float64       
 3   거주O_남성_0_work    1806 non-null   float64       
 4   거주O_남성_0_visit   1806 non-null   float64       
 5   거주O_남성_20_resd   1806 non-null   float64       
 6   거주O_남성_20_work   1806 non-null   float64       
 7   거주O_남성_20_visit  1806 non-null   float64       
 8   거주O_남성_30_resd   1806 non-null   float64       
 9   거주O_남성_30_work   1806 non-null   float64       
 10  거주O_남성_30_visit  1806 non-null   float64       
 11  거주O_남성_40_resd   1806 non-null   float64       
 12  거주O_남성_40_work   1806 non-null   float64       
 13  거주O_남성_40_visit  1806 non-null   float64       
 14  거주O_남성_60_resd   1806 non-null   float64